![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [1]:
import os
from ibm_watsonx_ai import APIClient, Credentials
from ibm_granite_community.notebook_utils import get_env_var
import getpass

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=get_env_var("IBM_CLOUD_TOKEN")
)



IBM_CLOUD_TOKEN loaded from Google Colab secret.


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [2]:
model_id = "ibm/granite-3-3-8b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [3]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0.7,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [4]:
project_id = get_env_var("PROJECT_ID")
space_id = get_env_var("SPACE_ID")


PROJECT_ID loaded from Google Colab secret.
SPACE_ID loaded from Google Colab secret.


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [5]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
	model_id = model_id,
	params = parameters,
	credentials = credentials,
	project_id = project_id,
	space_id = space_id
	)


## Defining the vector index
Initialize the vector index (not applicable here) to query when chatting with the model.

In [6]:
from ibm_watsonx_ai.foundation_models.utils import Toolkit

def proximity_search( query ):

    api_client = APIClient(
        project_id=project_id,
        credentials=credentials,
    )

    document_search_tool = Toolkit(
        api_client=api_client
    ).get_tool("RAGQuery")

    config = {
    "projectId": project_id
    }

    results = document_search_tool.run(
        input=query,
        config=config
    )

    return results.get("output")

## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [7]:
tone =  """informative"""

prompt_input = f"""Answer the provided question in a short, concise manner and a/an {tone} tone, but do not eliminate important content for the sake of brevity. Answer in paragraph form like the provided examples. There should be no space preceding the first word on the first line of a paragraph."""

chat_messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Q: How do I make a budget?\nA:"}
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "First, sit down and track all of your income and spending over the next few months. Once you have a full picture of your budget, you can start making changes. You'll get an idea of what expenses you can cut and where you can save a little extra or pay toward any debt you have."}
        ]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Q: Should I have an emergency fund?\nA:"}
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "An emergency fund is there to help you in the event something unexpected happens. If you lose your job or get hit with a big bill, you can use your emergency fund to help get you through until things are back on track. How much should you save in an emergency fund? Some financial professionals recommend having enough in an emergency fund to cover your expenses for at least three to six months, while others say you need enough to cover expenses for up to a year. Ultimately, it is up to you and depends on what you are able to put aside after taking care of necessary expenses. Having any amount ready to help when something unexpected happens is better than nothing."}
        ]
    },{
        "role": "user",
        "content": [
            {"type": "text", "text": "Q: Should I pay down my debts or save for retirement?\nA:"}
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "The answer to this question really depends on your financial situation. Your budget can provide some insight. In general, if you can start saving for retirement with a 401(k) through your employer, it's a good idea to start building retirement savings there. Then, look at your budget and see where you can make some cuts so you can also pay down debt. You may consider working with a financial professional to create a plan for paying off debt and saving for retirement."}
        ]
    },{
        "role": "user",
        "content": [
            {"type": "text", "text": "Q: Does my credit score matter?\nA:"}
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": """Your credit score can have a big impact on your life. A low credit score usually means you're offered less attractive terms and rates on personal loans, car loans and mortgages. The better your credit score, the more likely you are to save with more attractive rates.

Maintaining a good credit score is something you should start working on as soon as you can. Paying off debt and not having high balances is one way you can start to improve your score."""}
        ]
    },{
        "role": "user",
        "content": [
            {"type": "text", "text": "Q: How much money should you save each month?\nA:"}
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": """The amount you should save each month can vary based on your income, expenses, and financial goals. A common guideline is the 50/30/20 rule, which suggests allocating 50% of your income to necessities, 30% to discretionary expenses and 20% to savings and investments.

If you have specific saving goals, such as a down payment on a home or an early retirement, you may need to adjust these percentages to accelerate your savings rate."""}
        ]
    },{
        "role": "user",
        "content": [
            {"type": "text", "text": "Q: How do I set achievable targets for growing my savings over time?\nA:"}
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "Begin by analyzing your current cash flow: subtract your fixed expenses from your take-home pay to see what’s available. Break your extra cash into three buckets—savings, spending, and investing—and aim to direct at least one‐fifth of each paycheck toward your future goals. Translate that into monthly or weekly goals (e.g., $250 to savings each paycheck). As income rises or expenses change, revisit and adjust your targets to remain challenging yet realistic. Tracking progress visually—through a spreadsheet or goal-tracking app—helps you stay motivated and accountable."}
        ]
    },{
        "role": "user",
        "content": [
            {"type": "text", "text": "Q: What’s a reliable framework for translating income and expenses into a workable budget?\nA:"}
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "Try the “50/30/20” split: half your take-home pay on essentials, 30% on discretionary treats, and 20% toward debt and savings. To implement it, tally your average monthly net pay, categorize past expenses, and adjust until each bucket fits the guideline. Customize percentages to suit your goals—like boosting savings to 25% if you’re targeting aggressive debt payoff—and then automate transfers into separate accounts aligned with each category. Review monthly to fine-tune and ensure you’re living within your means."}
        ]
    }
]


## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [9]:
question = input("Question: ")
# grounding = proximity_search(question) # Removed the call to proximity_search
chat_messages.append({"role": "user", "content": question}) # Directly use the question
generated_response = model.chat(messages=chat_messages)
print(generated_response)

Question: How can I better budget my personal finances as a college student? How can I still pay down my student debt and save while in school?
{'id': 'chatcmpl-05af33b8571e8813427a3a88faeb4faa---6f2bc041-3406-4d17-93a2-ded1ddc7ad77', 'object': 'chat.completion', 'model_id': 'ibm/granite-3-3-8b-instruct', 'model': 'ibm/granite-3-3-8b-instruct', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Budgeting as a college student can be challenging, but with some planning and discipline, it's achievable. Here are some steps to help you budget effectively while managing student debt and saving:\n\n1. **Track your income and expenses**: Start by listing all your sources of income, including any part-time jobs, financial aid, or support from family. Then, identify and categorize your expenses into needs (rent, food, transportation, tuition) and wants (entertainment, non-essential shopping).\n\n2. **Create a budget plan**: Based on your income and expenses, allocate funds to 

In [10]:
output = generated_response['choices'][0]['message']['content']

In [11]:
from IPython.display import Markdown, display

def print_markdown(string):
  display(Markdown(string))

print_markdown(output)

Budgeting as a college student can be challenging, but with some planning and discipline, it's achievable. Here are some steps to help you budget effectively while managing student debt and saving:

1. **Track your income and expenses**: Start by listing all your sources of income, including any part-time jobs, financial aid, or support from family. Then, identify and categorize your expenses into needs (rent, food, transportation, tuition) and wants (entertainment, non-essential shopping).

2. **Create a budget plan**: Based on your income and expenses, allocate funds to each category. A modified version of the 50/30/20 rule (50% on needs, 30% on wants, 20% on savings and debt repayment) can be practical, though you might need to adjust percentages depending on your situation.

3. **Minimize unnecessary expenses**: Look for ways to cut down on non-essential costs. This could include eating out less, opting for more affordable entertainment options, using public transportation, or shopping around for the best deals on textbooks and other school supplies.

4. **Prioritize student loan repayment**: Make sure to include regular payments toward your student loans in your budget plan. Even small, consistent payments can help reduce your overall debt burden over time. Consider exploring options like income-driven repayment plans or refinancing if you qualify.

5. **Set up a small emergency fund**: Life can be unpredictable, and having a small financial cushion can help you manage unexpected expenses. Aim to save even a small amount each month.

6. **Explore additional income sources**: Look for ways to earn extra money to accelerate your savings and debt repayment. Consider part-time jobs, freelancing, or taking advantage of campus employment opportunities.

7. **Utilize student resources**: Many colleges offer financial counseling and budgeting workshops, so take advantage of these resources to help you develop effective money management skills.

8. **Save for post-graduation goals**: Even if your savings contributions seem small, start setting aside money for future needs like internships, job searching, or initial housing costs after graduation.

Remember, budgeting is an ongoing process and should be revisited and adjusted as your circumstances change. Stay committed and be patient; developing good financial habits in college will pay dividends long after you graduate.

# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  